In [15]:
import matplotlib.pyplot as plt
tf.compat.v1.disable_v2_behavior()
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

import os
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from alibi.explainers import CounterfactualProto

In [16]:
california = fetch_california_housing(as_frame=True)
X = california.data.to_numpy()
target = california.target.to_numpy()
feature_names = california.feature_names

In [17]:
california.data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [18]:
y = np.zeros((target.shape[0],))
y[np.where(target > np.median(target))[0]] = 1

In [19]:
mu = X.mean(axis=0)
sigma = X.std(axis=0)
X = (X - mu) / sigma

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [21]:
np.random.seed(42)
tf.random.set_seed(42)

In [22]:
def nn_model():
    x_in = Input(shape=(8,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [23]:
nn = nn_model()
nn.summary()
nn.fit(X_train, y_train, batch_size=64, epochs=30, verbose=1)
nn.save('nn_california.h5', save_format='h5')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_3 (Dense)             (None, 40)                360       
                                                                 
 dense_4 (Dense)             (None, 40)                1640      
                                                                 
 dense_5 (Dense)             (None, 2)                 82        
                                                                 
Total params: 2,082
Trainable params: 2,082
Non-trainable params: 0
_________________________________________________________________
Train on 16512 samples
Epoch 1/30
16512/16512 [==============================] - 0s 15us/sample - loss: 0.6445 - acc: 0.6779
Epoch 2/30
16512/16512 [==============================] - 0s 8us

In [25]:
nn = load_model('nn_california.h5')
score = nn.evaluate(X_test, y_test, verbose=0)
print('Test accuracy: ', score[1])

Test accuracy:  0.8498062


In [26]:
X = X_test[1].reshape((1,) + X_test[1].shape)
shape = X.shape

In [13]:
nn = load_model('nn_california.h5')

cf = CounterfactualProto(nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(X_train.min(axis=0), X_train.max(axis=0)), 
                         c_init=1., c_steps=10)

cf.fit(X_train)

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
No encoder specified. Using k-d trees to represent class prototypes.


CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 1.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (array([-1.77429947, -2.19618048, -1.83504572, -1.61076772, -1.25612255,
       -0.22899997, -1.44288613, -2.38599234]), array([  5.85828581,   1.85618152,  55.16323628,  51.78248741,
        30.25033022, 119.41910319,   2.95806762,   2.62528006])),
              'shape': (1, 8),
              'is_model': True,
              '

In [14]:
# generate a counterfactual
explanation = cf.explain(X)
print(explanation.data.keys())

dict_keys(['cf', 'all', 'orig_class', 'orig_proba', 'id_proto'])


In [28]:
print(f'Original prediction: {explanation.orig_class}')
print(f'Counterfactual prediction: {explanation.cf["class"]}')

Original prediction: 0
Counterfactual prediction: 1


In [32]:
orig = X * sigma + mu
counterfactual = explanation.cf['X'] * sigma + mu
delta = counterfactual - orig
print(delta)
for i, f in enumerate(feature_names):
    if np.abs(delta[0][i]) > 1e-2:
        print(f'{f}: {delta[0][i]}')#남쪽으로

[[-4.48098820e-08  3.79467160e-08 -1.79144086e-08  2.23376229e-09
   3.66006270e-06  7.68842279e-09 -1.91754168e-10 -1.19414201e-01]]
Longitude: -0.11941420102439793


In [33]:
pd.DataFrame(orig, columns=feature_names)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46


In [34]:
pd.DataFrame(counterfactual, columns=feature_names)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.5313,30.0,5.039384,1.193493,1565.000004,2.679795,35.14,-119.579414


In [35]:
os.remove('nn_california.h5')